In [1]:
import tensorflow as tf
import os
tf.random.set_seed(42)
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D, Input,LeakyReLU
# import tensorflow as tf
from tensorflow.keras import layers, models, Input
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras import backend as K
from tensorflow.data import AUTOTUNE
import os

In [2]:
# Defining Path
from google.colab import drive
drive.mount('/content/drive')
# DATASET_PATH = "/content/drive/MyDrive/dataset"
TRAINSET_PATH = "/content/drive/MyDrive/dataset/train"
VALSET_PATH = "/content/drive/MyDrive/dataset/val"
TESTSET_PATH ="/content/drive/MyDrive/dataset/test"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Define image size and batch size
IMAGE_SIZE = (224,224) # Resize all images to 224x224 pixel values
BATCH_SIZE = 32        # Number of images per batch

In [4]:
# Load dataset from directory
train_dataset = image_dataset_from_directory(
    TRAINSET_PATH,
    labels='inferred', # Infer labels from folder names
    label_mode="int",
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)
val_dataset = image_dataset_from_directory(
    VALSET_PATH,
    labels='inferred', # Infer labels from folder names
    label_mode="int",
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)
test_dataset = image_dataset_from_directory(
    TESTSET_PATH,
    labels='inferred', # Infer labels from folder names
    label_mode="int",
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

Found 5798 files belonging to 5 classes.
Found 826 files belonging to 5 classes.
Found 1656 files belonging to 5 classes.


In [5]:
# Split the dataset into training, validation, and test sets
# train_size = 0.7
# val_size = 0.2
# test_size = 0.1

In [6]:
# Getting the number of batches
# total_batches = tf.data.experimental.cardinality(dataset).numpy()
# train_batches = int(total_batches * train_size)
# val_batches = int(total_batches * val_size)
# print(total_batches,train_batches,val_batches)

In [7]:
# Split the data
# train_dataset = dataset.take(train_batches)
# remaining = dataset.skip(train_batches)
# val_dataset = remaining.take(val_batches)
# test_dataset = remaining.skip(val_batches)

In [8]:
for image, label in train_dataset.take(1):
    print(f"Image shape: {image.shape}, Label: {label.numpy()}")

Image shape: (32, 224, 224, 3), Label: [0 2 2 0 1 1 1 3 0 0 1 0 0 0 0 2 3 1 3 1 3 1 0 0 0 2 3 0 3 0 2 0]


In [9]:
print(f"Label shape: {label.shape}, Unique labels: {set(label.numpy())}")

Label shape: (32,), Unique labels: {np.int32(0), np.int32(1), np.int32(2), np.int32(3)}


In [10]:
# Normalize the pizel values to [0,1]
def normalize(image,label):
    image = tf.cast(image,tf.float32) / 255.0
    return image,label

import tensorflow.keras.backend as K


In [11]:
# Define Focal Loss function
def focal_loss(alpha=0.25, gamma=2.0):
    def loss(y_true, y_pred):
        y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=5)  # One-hot encode labels
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0)  # Avoid log(0)
        ce = -y_true * tf.math.log(y_pred)  # Cross-entropy
        weight = alpha * tf.pow(1 - y_pred, gamma)  # Focal weight
        return tf.reduce_sum(weight * ce, axis=1)  # Sum across classes
    return loss  # ✅ Return function (not a value)



In [12]:
import tensorflow as tf
from tensorflow.keras import layers,models

In [13]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),  # Randomly flip images horizontally
    layers.RandomRotation(0.1),       # Rotate images by ±10%
    layers.RandomZoom(0.1),           # Zoom-in or zoom-out by 10%
    layers.RandomContrast(0.1)        # Adjust contrast slightly
])
# ✅ Apply Augmentation ONLY to Training Dataset
train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x, training=True), y),
                                  num_parallel_calls=AUTOTUNE)


In [14]:
# Prefetch for performance
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [15]:
# import tensorflow.keras.backend as K

# # Custom function to restrict output between 0 and 4
# def bounded_relu(x):
#     return K.clip(x, 0, 4)


In [16]:
# Define the input shape
INPUT_SHAPE = (224,224,3) # Image size with 3 color channels (RGB)

In [17]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Define class labels
class_labels = np.array([0, 1, 2, 3, 4])

# Define the number of images per class (given dataset distribution)
samples_per_class = np.array([2286, 1046, 1516, 757, 173])

# Compute class weights (inverse frequency)
class_weights = compute_class_weight(class_weight="balanced", classes=class_labels, y=np.repeat(class_labels, samples_per_class))

# Convert to dictionary
class_weights_dict = {i: class_weights[i] for i in range(len(class_labels))}

# ✅ Reduce the max weight to prevent overcompensation
max_weight = 3.0  # ⬇ Limit max weight (previously 6.67)
class_weights_dict = {i: min(w, max_weight) for i, w in class_weights_dict.items()}

print("Final Adjusted Class Weights:", class_weights_dict)



Final Adjusted Class Weights: {0: np.float64(0.5055118110236221), 1: np.float64(1.1047801147227534), 2: np.float64(0.7622691292875989), 3: np.float64(1.5265521796565389), 4: 3.0}


In [ ]:


# ✅ Define bounded ReLU function
def bounded_relu(x):
    return K.relu(x, max_value=4)

# ✅ Register the function
get_custom_objects()['bounded_relu'] = bounded_relu

# ✅ Define input shape
INPUT_SHAPE = (224, 224, 3)

# ✅ Load EfficientNetB0 (without top layers)
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)
base_model.trainable = False  # Freeze pretrained layers

# ✅ Function to build the model
def build_model():
    inputs = Input(shape=INPUT_SHAPE)
    # Applying Data aggumentation
    x = data_augmentation(inputs)
    # Feature extraction using EfficientNet
    x = base_model(inputs, training=False)
    x = layers.Dropout(0.3)(x)

    # Additional CNN layers
    x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
    x = LeakyReLU(alpha=0.1)(x)  # ✅ Apply LeakyReLU
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = LeakyReLU(alpha=0.1)(x)  # ✅ Apply LeakyReLU
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)
    x = LeakyReLU(alpha=0.1)(x)  # ✅ Apply LeakyReLU

    # Flatten and fully connected layers
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation="relu")(x)

    # Classification output (5 classes)
    class_output = layers.Dense(5, activation='softmax', name='classification_output')(x)

    # Regression output (bounded ReLU for severity grading)
    # reg_output = layers.Dense(1, activation=bounded_relu, name='bounded_regression_output')(x)

    # Define the model
    model = models.Model(inputs=inputs, outputs=class_output)
    return model

# ✅ Build the model
model = build_model()

# ✅ Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss={
        "classification_output": focal_loss()
    },
    metrics={
        "classification_output": ["accuracy"],
    }
)



from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Define class labels
class_labels = np.array([0, 1, 2, 3, 4])

# Define the number of images per class
samples_per_class = np.array([2286, 1046, 1516, 757, 173])

# Compute class weights (inverse frequency)
class_weights = compute_class_weight(class_weight="balanced", classes=class_labels, y=np.repeat(class_labels, samples_per_class))

# Convert to dictionary
class_weights_dict = {i: class_weights[i] for i in range(len(class_labels))}

print("Computed Class Weights:", class_weights_dict)

# ✅ Convert class weights to float
class_weights_dict = {int(k): float(v) for k, v in class_weights_dict.items()}

print("Converted Class Weights:", class_weights_dict)  # ✅ Debugging


# ✅ Train model for 20 epochs
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=2,
    class_weight=class_weights_dict
)

Computed Class Weights: {0: np.float64(0.5055118110236221), 1: np.float64(1.1047801147227534), 2: np.float64(0.7622691292875989), 3: np.float64(1.5265521796565389), 4: np.float64(6.679768786127168)}
Converted Class Weights: {0: 0.5055118110236221, 1: 1.1047801147227534, 2: 0.7622691292875989, 3: 1.5265521796565389, 4: 6.679768786127168}
Epoch 1/2


/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


182/182 ━━━━━━━━━━━━━━━━━━━━ 607s 3s/step - accuracy: 0.2746 - loss: 0.2295 - val_accuracy: 0.4600 - val_loss: 0.1635
Epoch 2/2
182/182 ━━━━━━━━━━━━━━━━━━━━ 585s 3s/step - accuracy: 0.3698 - loss: 0.1679 - val_accuracy: 0.5254 - val_loss: 0.1482


In [ ]:
from google.colab import drive
drive.mount('/content/drive')  # Mounts Google Drive
model.save('/content/drive/My Drive/my_model_7.keras')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model.evaluate(test_dataset)

52/52 ━━━━━━━━━━━━━━━━━━━━ 121s 2s/step - accuracy: 0.4940 - loss: 0.1562


[0.15091250836849213, 0.5132850408554077]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ efficientnetb0 (Functional)          │ (None, 7, 7, 1280)          │       4,049,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 7, 7, 1280)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 7, 7, 32)            │         368,672 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 7, 7, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 3, 3, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 3, 3, 64)            │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 3, 3, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 1, 1, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 1, 1, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 1, 1, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ classification_output (Dense)        │ (None, 5)                   │             645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,484,116 (20.92 MB)

 Trainable params: 478,181 (1.82 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

 Optimizer params: 956,364 (3.65 MB)

In [ ]:
data set has folloing no. of images in training dataset from 0 to 4 level of severity 2286,1046,1516,757,173